In [1]:
import torch
torch.backends.cuda.matmul.allow_tf32 = True

import nip
from transformers import Trainer, TrainingArguments
import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib


from bitsandbytes.optim.adamw import AdamW as AdamW8bit
from torch.optim.adamw import AdamW
from transformers.optimization import Adafactor


DEVICE = 'cuda:0'

In [2]:
path_to_model = '/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/trellis/T256_L12_V1_K2_1mad_qtip.pth'
qmodel = torch.load(path_to_model)
fp_model = qlib.load_model('Llama2-7b-hf', torch_dtype=torch.float16)
tokenizer = qlib.load_tokenizer('Llama2-7b-hf')

/tmp/ipykernel_73862/3134161150.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  qmodel = torch.load(path_to_model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from torch.utils.data import Dataset, DataLoader

class KnowledgeDistillationDataset(Dataset):
    def __init__(self, data):
        self.data = data
        #self.lm_head = lm_head.cpu() #.to(DEVICE)
        self.len = len(self.data['decoder_output'])

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        assert index < self.len
        input_ids = self.data['input'][index]
        decoder_output = self.data['decoder_output'][index]
        return {
            'input_ids' : input_ids[0],
            'decoder_output' : decoder_output[0]
        }

kd_data_path = '/mnt/ssd_storage/ml/weights/vc_data/Llama2-7b-hf/kd_data'
dataset_name = 'kd_data_redpajama_decoder_output_small'
kd_data = torch.load(
    f'{kd_data_path}/{dataset_name}.pth', 
    weights_only=True
)

kd_dataset = KnowledgeDistillationDataset(kd_data)

In [4]:
bias_params = []

with torch.no_grad():
    qmodel.half()
    for param_name, param in qmodel.named_parameters():
        if 'bias' in param_name:
            bias_params.append(param)
            param.requires_grad = True


optimizer_cls = Adafactor
optimizer_kwargs = {
    'optimizer_dict': [
        {
            'params': bias_params,
            'lr': 1e-4, 
            'weight_decay': 0.0
        },
    ]
}

In [5]:
from torch.utils.checkpoint import checkpoint
qmodel.cuda()
qmodel.train()
qmodel._set_gradient_checkpointing(enable=True, gradient_checkpointing_func=checkpoint)

In [6]:
training_args = TrainingArguments(
    max_steps=32,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    
    logging_strategy="steps",
    logging_steps=1,
    output_dir = './output_dir',
    save_strategy="no",
    
    # label_names=[],
    # per_device_eval_batch_size=1,
    # eval_strategy='steps',
    # eval_steps=10,
    # eval_on_start=True,

    remove_unused_columns=False,
)


lm_head = fp_model.lm_head.to(DEVICE)

class CustomTrainer(Trainer):
    # def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
    #     outputs = model(inputs['input_ids'], labels=inputs['input_ids'])
    #     loss = outputs.loss
    #     return (loss, outputs) if return_outputs else loss
    
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        outputs = model(inputs['input_ids'], labels=inputs['input_ids'])
        ce_loss = outputs.loss

        qmodel_logits = outputs.logits
        fpmodel_logits = lm_head(inputs['decoder_output'])
        
        n_tokens = torch.prod(torch.tensor(qmodel_logits.shape[:-1]))
        
        T = 2
        kd_loss = torch.nn.functional.kl_div(
                torch.log_softmax(qmodel_logits / T, dim=1),
                torch.softmax(fpmodel_logits / T, dim=1),
                reduction='batchmean',
            ) * (T**2) / n_tokens
        
        # print("kd_loss:", kd_loss.item(), 'ce_loss:', ce_loss.item())
        # sim = torch.argmax(qmodel_logits, dim=-1) == torch.argmax(fpmodel_logits, dim=-1)
        # print('right_labels ratio:', sim.sum().item() / n_tokens)

        total_loss = ce_loss + 0.1 * kd_loss

        return (total_loss, outputs) if return_outputs else total_loss


trainer = CustomTrainer(
    model=qmodel,
    args=training_args,
    #train_dataset=train_dataset,
    train_dataset=kd_dataset,
    #eval_dataset=eval_dataset,
    optimizer_cls_and_kwargs=(optimizer_cls, optimizer_kwargs)
)

trainer.can_return_loss = True
with torch.amp.autocast('cuda', dtype=torch.float16):
    trainer.train()

Step,Training Loss
1,53.023200
2,48.466200
3,51.541700
4,50.996700


KeyboardInterrupt: 